In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch
from src.utils.datasets.ggimages import OpenImage
from src.utils.datasets.transform import RandomHorizontalFlip, Resize, Compose, XyToCenter
import torchvision.transforms as transforms
from src.utils.display.images import imshow, result_show
from torch.utils.data import DataLoader
from src.utils.datasets.adapter import convert_data

from src.network.yolo import Yolo

general_transform = Compose([
    Resize((608, 608)),
    RandomHorizontalFlip(0.5),
    XyToCenter()
])


transform = transforms.Compose([
                transforms.RandomChoice([
                    transforms.ColorJitter(hue=.3, saturation=.2),
                    transforms.RandomGrayscale(p=0.3),
                ]),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [
                                     0.229, 0.224, 0.225])
            ])

In [ ]:
batch_size = 4

ds = OpenImage('/data/data/SmallDataset/', 'train', general_transform=general_transform, transform=transform)

train_data_loader = DataLoader(ds, batch_size=batch_size, shuffle=True, collate_fn=convert_data, num_workers=0)

In [ ]:
ds.classes

In [ ]:
for i, data in enumerate(train_data_loader):
    if not data:
        continue
    if i > 3:
        break
    batch_tensor, batch_boxes, detectors_mask, matching_true_boxes, im_info, img_names = data
    #     print batch_boxes.shape
    for k in range(batch_tensor.shape[0]):
        current_im_info = im_info[0]
        tmp = batch_boxes[k] * torch.Tensor([current_im_info[0], current_im_info[1], current_im_info[0], current_im_info[1], 1])
        tmp = tmp.numpy()        
        between = tmp[:, 2:4] / 2        
        xy = tmp[:, :2]
        xy_min = xy - between
        xy_max = xy + between
        imshow(batch_tensor[k], gt_boxes=np.hstack((xy_min, xy_max)))